In [ ]:
import numpy as np

import section4_7 as hwh
import section7_1 as dhw
import section8_a as spec
import section8_d as bb_dhw
import section8_e as bb_hwh

from section11_1 import load_outdoor, get_Theta_ex

# 第八章　コージェネレーション設備

1．適用範囲～4．記号及び単位は省略

### 5. ガス消費量

$$\large
\displaystyle
E_{G,CG,d,t} = E_{G,PU,d,t} + E_{G,BB,DHW,d,t} + E_{G,BB,HWH,d,t}
$$

$E_{G,CG,d,t}$：日付$d$の時刻$t$における1時間当たりのコージェネレーション設備のガス消費量(MJ/h)  
$E_{G,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットのガス消費量(MJ/h)  
$E_{G,BB,DHW,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯時のバックアップボイラーのガス消費量(MJ/h)  
$E_{G,BB,HWH,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房時のバックアップボイラーのガス消費量(MJ/h)  

In [ ]:
# 1時間当たりのコージェネレーション設備のガス消費量 (1)
def get_E_G_CG_d_t(bath_function, CG, E_E_dmd_d_t,
                   L_dashdash_k_d_t, L_dashdash_w_d_t, L_dashdash_s_d_t, L_dashdash_b1_d_t, L_dashdash_b2_d_t,
                   L_dashdash_ba1_d_t,
                   L_dashdash_ba2_d_t,
                   H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_rad):
    """
    :param bath_function: ふろ機能の種類
    :param CG: コージェネレーション設備の仕様
    :param E_E_dmd_d_t: 1時間当たりの電力需要 (kWh/h)
    :param L_HWH_d_t: 1時間当たりの温水暖房の熱負荷 (MJ/h)
    :param r_WS_HWH_d_t: 温水暖房の温水供給運転率 (-)
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚の太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりのコージェネレーション設備の一次エネルギー消費量及び1時間当たりのコージェネレーション設備による発電量
    """
    # ----- パラメータの取得 -----

    if 'CG_category' in CG:
        # 温水暖房への排熱利用
        exhaust = spec.get_exhaust(CG['CG_category'])

        # 排熱利用方式
        exhaust_priority = spec.get_exhaust_priority(CG['CG_category'])

        # バックアップボイラー(給湯)の熱源種別
        type_BB_DHW = spec.get_type_BB_DHW(CG['CG_category'])

        # バックアップボイラー(給湯)の給湯機の効率
        e_rtd_DHW_BB = spec.get_e_rtd_BB_DHW(CG['CG_category'])

        # バックアップボイラー(温水暖房)の種類
        type_BB_HWH = spec.get_type_BB_HWH(CG['CG_category'])

        # バックアップボイラー(温水暖房)の定格効率
        e_rtd_BB_HWH = spec.get_e_rtd_BB_HWH(CG['CG_category'])

        # バックアップボイラー(温水暖房)の定格能力 (W)
        q_rtd_BB_HWH = spec.get_q_rtd_BB_HWH(CG['CG_category'])

        # 発電ユニットの給湯排熱利用率
        r_DHW_gen_PU_d = spec.get_r_DHW_gen_PU_d(CG['CG_category'])

        # 発電ユニットの温水暖房排熱利用率
        r_HWH_gen_PU_d = spec.get_r_HWH_gen_PU_d(CG['CG_category'])

        # 発電ユニットの発電方式
        PU_type = spec.get_PU_type(CG['CG_category'])

        # 発電ユニットの発電量推定時の仮想発電量のパラメータ a_PU, a_DHW, a_HWH, b, c
        param_E_E_gen_PU_Evt_d = spec.get_param_E_E_gen_PU_EVt_d(CG['CG_category'])

        # 発電ユニットの排熱量推定時の仮想燃料消費量を求める係数
        param_E_F_PU_HVt_d = spec.get_param_E_F_PU_HVt_d(CG['CG_category'])

        # 発電ユニットの排熱量推定時の仮想排熱量の上限比を求める係数 a_DHW, a_HWH, b
        param_r_H_gen_PU_HVt_d = spec.get_param_r_H_gen_PU_HVt_d(CG['CG_category'])

        # 発電ユニットの日平均発電効率を求める係数 a_PU, a_DHW, a_HWH, b, 上限値, 下限値
        param_e_E_PU_d = spec.get_param_e_E_PU_d(CG['CG_category'])

        # 発電ユニットの日平均排熱効率を求める係数 a_PU, a_DHW, a_HWH, b, 上限値, 下限値
        param_e_H_PU_d = spec.get_param_e_H_PU_d(CG['CG_category'])

        # 定格発電出力 (W)
        P_rtd_PU = spec.get_P_rtd_PU(CG['CG_category'])

        # タンクユニットの補機消費電力 (給湯)
        P_TU_aux_DHW = spec.get_P_TU_aux_DHW(CG['CG_category'])

        # タンクユニットの補機消費電力 (温水暖房)
        P_TU_aux_HWH = spec.get_P_TU_aux_HWH(CG['CG_category'])

        # 逆潮流の評価
        has_CG_reverse = CG['reverse'] if 'reverse' in CG else False

    else:
        # 温水暖房への排熱利用
        exhaust = CG['exhaust']

        # 排熱利用方式
        exhaust_priority = CG['exhaust_priority']

        # バックアップボイラー(給湯)の熱源種別
        type_BB_DHW = CG['type_BB_DHW']

        # バックアップボイラー(給湯)の給湯機の効率
        e_rtd_DHW_BB = CG['e_rtd_DHW_BB']

        # バックアップボイラー(温水暖房)の種類
        type_BB_HWH = CG['type_BB_HWH']

        # バックアップボイラー(温水暖房)の定格効率
        e_rtd_BB_HWH = CG['e_rtd_BB_HWH']

        # バックアップボイラー(温水暖房)の定格能力 (W)
        q_rtd_BB_HWH = CG['q_rtd_BB_HWH']

        # 発電ユニットの給湯排熱利用率
        r_DHW_gen_PU_d = CG['r_DHW_gen_PU_d']

        # 発電ユニットの温水暖房排熱利用率
        r_HWH_gen_PU_d = CG['r_HWH_gen_PU_d']

        # 発電ユニットの発電方式
        PU_type = CG['PU_type']

        # 発電ユニットの発電量推定時の仮想発電量のパラメータ a_PU, a_DHW, a_HWH, b, c
        param_E_E_gen_PU_Evt_d = CG['param_E_E_gen_PU_Evt_d']

        # 発電ユニットの排熱量推定時の仮想燃料消費量を求める係数
        param_E_F_PU_HVt_d = CG['param_E_F_PU_HVt_d']

        # 発電ユニットの排熱量推定時の仮想排熱量の上限比を求める係数 a_DHW, a_HWH, b
        param_r_H_gen_PU_HVt_d = CG['param_r_H_gen_PU_HVt_d']

        # 発電ユニットの日平均発電効率を求める係数 a_PU, a_DHW, a_HWH, b, 上限値, 下限値
        param_e_E_PU_d = CG['param_e_E_PU_d']

        # 発電ユニットの日平均排熱効率を求める係数 a_PU, a_DHW, a_HWH, b, 上限値, 下限値
        param_e_H_PU_d = CG['param_e_H_PU_d']

        # 定格発電出力 (W)
        P_rtd_PU = CG['P_rtd_PU']

        # タンクユニットの補機消費電力 (給湯)
        P_TU_aux_DHW = CG['P_TU_aux_DHW']

        # タンクユニットの補機消費電力 (温水暖房)
        P_TU_aux_HWH = CG['P_TU_aux_HWH']

        # 逆潮流の評価
        has_CG_reverse = CG['reverse'] if 'reverse' in CG else False

    # ----- 温水暖房用熱源機の負荷および温水供給運転率の計算 -----

    if H_HS is not None and H_HS['type'] == 'コージェネレーションを使用する':
        # 主たる居室、その他の居室という単位で設定された放熱機器を暖房区画ごとの配列に変換
        rad_list = hwh.get_rad_list(H_MR, H_OR)

        # 温水暖房用熱源機の往き温水温度
        Theta_SW_hs_op = hwh.get_Theta_SW_hs_op(type_BB_HWH)
        p_hs = hwh.get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_rad, A_A, A_MR, A_OR, region, mode_MR, mode_OR)
        Theta_SW_d_t = hwh.get_Theta_SW_d_t(Theta_SW_hs_op, p_hs)

        # 1時間当たりの温水暖房の熱負荷 (MJ/h)
        L_HWH_d_t = hwh.get_Q_dmd_H_hs_d_t(rad_list, H_HS['pipe_insulation'], H_HS['underfloor_pipe_insulation'],
                                           Theta_SW_d_t, A_A, A_MR, A_OR, region,
                                           mode_MR, mode_OR, L_T_H_rad)

        # 処理暖房負荷
        Q_T_H_rad = np.zeros((5, 24 * 365))
        for i in [1, 3, 4, 5]:

            if rad_list[i - 1] is None:
                continue

            # 1時間当たりの暖冷房区画iに設置された放熱器の最大暖房出力
            A_HCZ = hwh.get_A_HCZ_i(i, A_A, A_MR, A_OR)
            R_type = '主たる居室' if i == 1 else 'その他の居室'
            mode = mode_MR if i == 1 else mode_OR
            Q_max_H_rad_d_t_i = hwh.get_Q_max_H_rad_d_t_i(rad_list[i - 1], A_HCZ, Theta_SW_d_t, region, mode, R_type)

            # 1時間当たりの暖冷房区画iに設置された放熱器の処理暖房負荷
            Q_T_H_rad[i - 1, :] = hwh.get_Q_T_H_rad_d_t_i(Q_max_H_rad_d_t_i, L_T_H_rad[i - 1])

        # 温水暖房用熱源機の温水供給運転率
        r_WS_HWH_d_t = hwh.get_r_WS_hs_d_t(rad_list, L_HWH_d_t, Q_T_H_rad, Theta_SW_d_t, region, A_A, A_MR, A_OR,
                                           mode_MR)
        # 戻り温水温度 (9)
        Theta_RW_hs = hwh.get_Theta_RW_hs_d_t(Theta_SW_d_t, rad_list, H_HS['pipe_insulation'],
                                              H_HS['underfloor_pipe_insulation'], A_A, A_MR, A_OR, region,
                                              mode_MR, mode_OR,
                                              L_T_H_rad)

        # 定格能力の計算のためのパラメータの取得
        rad_types = hwh.get_rad_type_list()
        has_MR_hwh = H_MR['type'] in rad_types
        if H_OR is not None:
            has_OR_hwh = H_OR['type'] in rad_types
        else:
            has_OR_hwh = False
    else:
        L_HWH_d_t = np.zeros(24 * 365)
        P_TU_aux_HWH = np.zeros(24 * 365)
        r_WS_HWH_d_t = np.zeros(24 * 365)

    # 外気温度の取得
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)
    Theta_ex_Ave = dhw.get_Theta_ex_Ave_d(Theta_ex_d_t)

    # ----- 16. その他 -----

    # 1日当たりの温水暖房の熱負荷 (31)
    L_HWH_d = get_L_HWH_d(L_HWH_d_t)

    # 1時間当たりの発電ユニットによる浴槽追焚を除く給湯熱負荷 (30)
    L_DHW_d_t = get_L_DHW_d_t(L_dashdash_k_d_t, L_dashdash_w_d_t, L_dashdash_s_d_t, L_dashdash_b1_d_t,
                              L_dashdash_b2_d_t, L_dashdash_ba1_d_t)

    # 1日当たりの発電ユニットによる浴槽追焚を除く給湯熱負荷 (MJ/d) (29)
    L_DHW_d = get_L_DHW_d(L_DHW_d_t)

    # ----- 15. タンクユニットの補機消費電力 -----

    # 1時間当たりの浴槽追焚時におけるバックアップボイラーが分担する給湯熱負荷 (MJ/h) (28)
    L_BB_DHW_ba2_d_t = get_L_BB_DHW_ba2_d_t(L_dashdash_ba2_d_t)

    # 1時間当たりの浴槽追焚のタンクユニットの補機消費電力量 (kWh/h)
    E_E_TU_aux_ba2_d_t = get_E_E_TU_aux_ba2_d_t(L_BB_DHW_ba2_d_t)

    # 1時間当たりの温水暖房時のタンクユニットの補機消費電力量 (kWh/h) (27)
    E_E_TU_aux_HWH_d_t = get_E_E_TU_aux_HWH_d_t(exhaust, P_TU_aux_HWH, r_WS_HWH_d_t)

    # 1時間当たりの給湯時のタンクユニットの補機消費電力量 (kWh/h) (26)
    E_E_TU_aux_DHW_d_t = get_E_E_TU_aux_DHW_d_t(P_TU_aux_DHW)

    # 1時間当たりのタンクユニットの補機消費電力量 (25)
    E_E_TU_aux_d_t = get_E_E_TU_aux_d_t(E_E_TU_aux_DHW_d_t, E_E_TU_aux_HWH_d_t, E_E_TU_aux_ba2_d_t)

    print('E_E_TU_aux_ba2_d_t = {} [kWh/yr] (min={}, max={})'.format(np.sum(E_E_TU_aux_ba2_d_t), min(E_E_TU_aux_ba2_d_t), max(E_E_TU_aux_ba2_d_t)))
    print('E_E_TU_aux_HWH_d_t = {} [kWh/yr] (min={}, max={})'.format(np.sum(E_E_TU_aux_HWH_d_t), min(E_E_TU_aux_HWH_d_t), max(E_E_TU_aux_HWH_d_t)))
    print('E_E_TU_aux_DHW_d_t = {} [kWh]'.format(E_E_TU_aux_DHW_d_t))
    print('E_E_TU_aux_d_t = {} [kWh/yr] (min={}, max={})'.format(np.sum(E_E_TU_aux_d_t), min(E_E_TU_aux_d_t), max(E_E_TU_aux_d_t)))
    np.savetxt('E_E_TU_aux_ba2_d_t_cogen.csv', E_E_TU_aux_ba2_d_t)
    np.savetxt('E_E_TU_aux_HWH_d_t_cogen.csv', E_E_TU_aux_HWH_d_t)

    # ----- 14. 発電ユニット -----

    # 発電ユニットの電力需要 (kWh/h) (24)
    E_E_dmd_PU_d_t = get_E_E_dmd_PU_d_t(E_E_dmd_d_t, E_E_TU_aux_d_t)

    # 1時間当たりの発電ユニットの分担可能電力負荷 (kWh/h) (23)
    E_E_PU_d_t = get_E_E_PU_d_t(E_E_dmd_PU_d_t, P_rtd_PU, has_CG_reverse)

    # 1日当たりの発電ユニットの分担可能電力負荷 (kWh/d) (22)
    E_E_PU_d = get_E_E_PU_d(E_E_PU_d_t)

    # 発電ユニットの日平均排熱効率 (-) (21)
    e_H_PU_d = get_e_H_PU_d(E_E_PU_d, L_DHW_d, L_HWH_d, **param_e_H_PU_d)

    # 発電ユニットの日平均発電効率 (-) (20)
    e_E_PU_d = get_e_E_PU_d(E_E_PU_d, L_DHW_d, L_HWH_d, **param_e_E_PU_d)

    if PU_type == '熱主':
        # 発電ユニットの排熱量推定時の仮想排熱量上限比 (-) (19)
        r_H_gen_PU_HVt_d = get_r_H_gen_PU_HVt_d(L_DHW_d, L_HWH_d, **param_r_H_gen_PU_HVt_d)

        # 1日当たりの発電ユニットの排熱量推定時の仮想燃料消費 (MJ/d) (18)
        E_F_PU_HVt_d = get_E_G_PU_HVt_d(e_H_PU_d, L_DHW_d, L_HWH_d, r_H_gen_PU_HVt_d, **param_E_F_PU_HVt_d)
    else:
        E_F_PU_HVt_d = np.zeros(365)

    # 1日当たりの発電ユニットの発電量推定時の仮想発電量 (kWh/d) (17)
    E_E_gen_PU_EVt_d = get_E_E_gen_PU_EVt_d(E_E_PU_d, L_DHW_d, L_HWH_d, **param_E_E_gen_PU_Evt_d)

    # 1日当たりの発電ユニットの発電量推定時の仮想ガス消費量 (MJ/d) (16)
    E_F_PU_EVt_d = get_E_G_PU_EVt_d(E_E_gen_PU_EVt_d, e_E_PU_d)

    # 1日当たりの発電ユニットのガス消費量 (MJ/d) (15)
    E_G_PU_d = get_E_G_PU_d(PU_type, E_F_PU_EVt_d, E_F_PU_HVt_d)

    # 1時間当たりの発電ユニットのガス消費量 (MJ/h) (14a)
    E_G_PU_d_t = calc_E_G_PU_d_t(E_G_PU_d, E_E_PU_d, E_E_PU_d_t, e_E_PU_d)

    # 発電ユニットの発電量 (kWh/h) (10)
    E_E_gen_PU_d_t = get_E_E_gen_PU_d_t(E_G_PU_d_t, e_E_PU_d)

    # 1日当たりの発電ユニット排熱量 (MJ/d) (9)
    Q_PU_gen_d = get_Q_PU_gen_d(E_G_PU_d, e_H_PU_d)

    # 1日当たりの給湯・温水暖房の排熱利用量 (MJ/d) (6)(7)(8)
    Q_gen_DHW_d, Q_gen_HWH_d = calc_Q_gen_x_d(exhaust, exhaust_priority, Q_PU_gen_d, r_DHW_gen_PU_d, r_HWH_gen_PU_d,
                                              L_DHW_d, L_HWH_d)

    print('E_E_dmd_PU_d_t = {} [MJ/yr] (min={}, max={} [MJ/h])'.format(np.sum(E_E_dmd_PU_d_t), min(E_E_dmd_PU_d_t), max(E_E_dmd_PU_d_t)))
    print('P_rtd_PU = {}'.format(P_rtd_PU))
    print('E_E_PU_d = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(E_E_PU_d), min(E_E_PU_d), max(E_E_PU_d)))
    print('E_E_gen_PU_EVt_d = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(E_E_gen_PU_EVt_d), min(E_E_gen_PU_EVt_d), max(E_E_gen_PU_EVt_d)))
    print('e_E_PU_d = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(e_E_PU_d), min(e_E_PU_d), max(e_E_PU_d)))
    print('E_F_PU_EVt_d = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(E_F_PU_EVt_d), min(E_F_PU_EVt_d), max(E_F_PU_EVt_d)))
    print('E_F_PU_HVt_d = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(E_F_PU_HVt_d), min(E_F_PU_HVt_d), max(E_F_PU_HVt_d)))
    print('E_G_PU_d = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(E_G_PU_d), min(E_G_PU_d), max(E_G_PU_d)))
    print('E_G_PU_d_t = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(E_G_PU_d_t), min(E_G_PU_d_t), max(E_G_PU_d_t)))
    print('E_E_gen_PU = {} [MJ/yr] (min={}, max={} [MJ/h])'.format(np.sum(E_E_gen_PU_d_t), min(E_E_gen_PU_d_t), max(E_E_gen_PU_d_t)))
    print('Q_PU_gen = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(Q_PU_gen_d), min(Q_PU_gen_d), max(Q_PU_gen_d)))
    print('Q_gen_DHW = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(Q_gen_DHW_d), min(Q_gen_DHW_d), max(Q_gen_DHW_d)))
    print('Q_gen_HWH = {} [MJ/yr] (min={}, max={} [MJ/d])'.format(np.sum(Q_gen_HWH_d), min(Q_gen_HWH_d), max(Q_gen_HWH_d)))

    # ----- 13. 温水暖房時のバックアップボイラーのガス消費量(温水暖房への排熱利用がある場合)

    if H_HS is not None and H_HS['type'] == 'コージェネレーションを使用する':
        # 1日当たりの給湯のバックアップボイラーが分担する給湯熱負荷 (MJ/d) (5)
        L_BB_HWH_d_t = get_L_BB_HWH_d_t(L_HWH_d_t, L_HWH_d, Q_gen_HWH_d)

        # 1時間当たりの温水暖房時のバックアップボイラーのガス消費量 (MJ/h)
        E_G_BB_HWH_d_t = bb_hwh.calc_E_G_BB_HWH_d_t(type_BB_HWH, e_rtd_BB_HWH, q_rtd_BB_HWH, L_BB_HWH_d_t, p_hs)
    else:
        L_BB_HWH_d_t = np.zeros(24 * 365)
        E_G_BB_HWH_d_t = np.zeros(24 * 365)

    # ----- 12. 給湯時のバックアップボイラーのガス消費量 ----

    # 1時間あたりのバックアップボイラーが分担する給湯熱負荷 (4)
    L_BB_DHW_d_t, L_BB_DHW_k_d_t, L_BB_DHW_s_d_t, L_BB_DHW_w_d_t, \
    L_BB_DHW_b1_d_t, L_BB_DHW_b2_d_t, L_BB_DHW_ba1_d_t, L_BB_DHW_ba2_d_t = \
        get_L_BB_x_d_t(L_dashdash_k_d_t, L_dashdash_w_d_t, L_dashdash_s_d_t,
                       L_dashdash_b1_d_t, L_dashdash_b2_d_t, L_dashdash_ba1_d_t, L_dashdash_ba2_d_t,
                       Q_gen_DHW_d, L_DHW_d)

    # 1日当たりの給湯時のバックアップボイラーの燃料消費量 (MJ/d) (3)
    E_G_BB_DHW_d_t, E_G_BB_DHW_ba2_d_t = bb_dhw.calc_E_G_BB_DHW_d_t(bath_function,
                                                L_BB_DHW_k_d_t, L_BB_DHW_s_d_t, L_BB_DHW_w_d_t,
                                                L_BB_DHW_b1_d_t, L_BB_DHW_b2_d_t, L_BB_DHW_ba1_d_t, L_BB_DHW_ba2_d_t,
                                                e_rtd_DHW_BB, Theta_ex_Ave)

    # ----- 11. 給湯時のバックアップボイラーの年間平均効率 -----

    # 給湯時のバックアップボイラーの年間平均効率 (-) (6)
    e_BB_ave = get_e_BB_ave(L_BB_DHW_d_t, L_BB_DHW_ba2_d_t, E_G_BB_DHW_d_t, E_G_BB_DHW_ba2_d_t)

    # ----- 10. 製造熱量のうちの自家消費算入分 -----

    # 1年あたりのコージェネレーション設備による製造熱量のうちの自家消費算入分 (MJ/yr) (5)
    Q_CG_h = get_Q_CG_h(L_DHW_d_t)

    # ----- 9. 発電量のうちの自己消費分

    # 1年当たりのコージェネレーション設備による発電量のうちの自己消費分 (kWH/yr) (4)
    E_E_CG_self = get_E_E_CG_self(E_E_TU_aux_d_t)

    # ----- 8. 給湯時のバックアップボイラーの年間平均効率 -----

    # 1年あたりのコージェネレーション設備のガス消費量のうちの売電に係る控除対象分 (MJ/yr) (3)
    E_G_CG_ded = get_E_G_CG_ded(E_G_PU_d_t, E_G_BB_DHW_d_t, E_G_BB_DHW_ba2_d_t)

    # ----- 7. 発電量 -----

    # 1時間当たりのコージェネレーション設備による発電量 (kWh/h) (2)
    E_E_CG_gen_d_t = get_E_E_CG_gen_d_t(E_E_gen_PU_d_t, E_E_TU_aux_d_t)

    # ----- 5. ガス消費量 -----

    # 1時間当たりのコージェネレーション設備のガス消費量 (MJ/h) (1)
    E_G_CG_d_t = E_G_PU_d_t + E_G_BB_DHW_d_t + E_G_BB_HWH_d_t

    print('E_G_PU = {} [MJ/yr]'.format(np.sum(E_G_PU_d_t)))
    print('E_G_BB_DHW = {} [MJ/yr]'.format(np.sum(E_G_BB_DHW_d_t)))
    print('E_G_BB_HWH = {} [MJ/yr]'.format(np.sum(E_G_BB_HWH_d_t)))
    print('E_E_CG_gen = {} [kWh/yr]'.format(np.sum(E_E_CG_gen_d_t)))
    print('E_G_CG = {} [MJ/yr]'.format(np.sum(E_G_CG_d_t)))

    return E_G_CG_d_t, E_E_CG_gen_d_t, E_G_CG_ded, E_E_CG_self, Q_CG_h, E_E_TU_aux_d_t, e_BB_ave

### 6. 灯油消費量

日付$d$の時刻$t$における1時間当たりのコージェネレーション設備の灯油消費量$E_{K,CG,d,t}$は、0とする。 

In [ ]:
# 1時間当たりのコージェネレーション設備の灯油消費量 (MJ/h)
def get_E_K_CG_d_t():
    return np.zeros(24 * 365)


### 7. 発電量

$$\large
\displaystyle
E_{E,CG,gen,d,t} = E_{E,gen,PU,d,t} - E_{E,TU,aux,d,t}
$$

$E_{E,CG,gen,d,t}$：日付$d$の時刻$t$における1時間当たりのコージェネレーション設備による発電量(kWh/h)  
$E_{E,gen,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットの発電量(kWh/h)  
$E_{E,TU,aux,d,t}$：日付$d$の時刻$t$における1時間当たりのタンクユニットの補機消費電力量(kWh/h)  

In [ ]:
# 1時間当たりのコージェネレーション設備による発電量 (kWh/h) (2)
def get_E_E_CG_gen_d_t(E_E_gen_PU_d_t, E_E_TU_aux_d_t):
    """
    :param E_E_gen_PU_d_t: 1時間当たりの発電ユニットの発電量 (kWh/h)
    :param E_E_TU_aux_d_t: 1時間当たりのタンクユニットの補機消費電力量 (kWh/h)
    :return: 1時間当たりのコージェネレーション設備による発電量 (kWh/h)
    """
    return E_E_gen_PU_d_t - E_E_TU_aux_d_t


### 8. ガス消費量のうちの売電に係る控除対象分

$$\large
\displaystyle
E_{G,CG,ded} = \sum^{365}_{d=1} \sum^{23}_{t=0} \bigg(E_{G,PU,d,t}+E_{G,BB,DHW,d,t}-E_{G,BB,DHW,ba2,d,t}\bigg)
$$

$E_{G,CG,ded}$：1 年当たりのコージェネレーション設備のガス消費量のうちの売電に係る控除対象分(MJ/yr)  
$E_{G,BB,DHW,ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚時におけるバックアップボイラーのガス消費量(MJ/h)  
$E_{G,BB,DHW,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯時のバックアップボイラーのガス消費量(MJ/h)  
$E_{G,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットのガス消費量(MJ/h)  

In [ ]:
# 1年あたりのコージェネレーション設備のガス消費量のうちの売電に係る控除対象分 (MJ/yr) (3)
def get_E_G_CG_ded(E_G_PU_d_t, E_G_BB_DHW_d_t, E_G_BB_DHW_ba2_d_t):
    """
    :param E_G_PU_d_t: 日付dの時刻tにおける1時間当たりの発電ユニットのガス消費量 (MJ/h)
    :param E_G_BB_DHW_d_t: 日付dの時刻tにおける1時間当たりの給湯時のバックアップボイラーのガス消費量 (MJ/h)
    :param E_G_BB_DHW_ba2_d_t: 日付dの時刻tにおける1時間当たりの浴槽追焚時におけるバックアップボイラーのガス消費量 (MJ/d)
    :return: 1年あたりのコージェネレーション設備のガス消費量のうちの売電に係る控除対象分 (MJ/yr)
    """
    return np.sum(E_G_PU_d_t + E_G_BB_DHW_d_t - E_G_BB_DHW_ba2_d_t)


### 9. 発電量のうちの自己消費分

$$\large
\displaystyle
E_{G,CG,self} = \sum^{365}_{d=1} \sum^{23}_{t=0} E_{E,TU,aux,d,t}
$$

$E_{G,CG,self}$：1 年当たりのコージェネレーション設備による発電量のうちの自己消費分(kWh/yr)  
$ E_{E,TU,aux,d,t}$：日付$d$の時刻$t$における1時間当たりのタンクユニットの補機消費電力量(kWh/h)  

In [ ]:
# 1年当たりのコージェネレーション設備による発電量のうちの自己消費分 (kWH/yr) (4)
def get_E_E_CG_self(E_E_TU_aux_d_t):
    """
    :param E_E_TU_aux_d_t: 日付dの時刻tにおける1時間当たりのタンクユニットの補機消費電力量 (kWh/yr)
    :return: 1年当たりのコージェネレーション設備による発電量のうちの自己消費分 (kWH/yr)
    """
    return np.sum(E_E_TU_aux_d_t)


### 10. 製造熱量のうちの自家消費算入分

$$\large
\displaystyle
Q_{CG,h} = \sum^{365}_{d=1} \sum^{23}_{t=0} L_{DHW,d,t}
$$

$Q_{CG,h}$：1 年当たりのコージェネレーション設備による製造熱量のうちの自家消費算入分(MJ/yr)  
$L_{DHW,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/h)  

In [ ]:
# 1年あたりのコージェネレーション設備による製造熱量のうちの自家消費算入分 (MJ/yr) (5)
def get_Q_CG_h(L_DHW_d_t):
    """
    :param L_DHW_d_t: 日付dの時刻tにおける1時間当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :return: 1年あたりのコージェネレーション設備による製造熱量のうちの自家消費算入分 (MJ/yr)
    """
    return np.sum(L_DHW_d_t)


### 11. 給湯時のバックアップボイラーの年間平均効率

$$\large
\displaystyle
e_{BB,ave} = \frac{\sum^{365}_{d=1} \sum^{23}_{t=0} (L_{BB,DHW,d,t}-L_{BB,DHW,ba2,d,t})}{\sum^{365}_{d=1} \sum^{23}_{t=0} (E_{G,BB,DHW,d,t}-E_{G,BB,DHW,ba2,d,t})}
$$

$e_{BB,ave}$：給湯時のバックアップボイラーの年間平均効率(-)  
$L_{BB,DHW,d,t}$：日付$d$の時刻$t$における1時間当たりのバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚時におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$E_{G,BB,DHW,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯時のバックアップボイラーのガス消費量(MJ/h)  
$E_{G,BB,DHW,ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚時におけるバックアップボイラーのガス消費量(MJ/h)  

In [ ]:
# 給湯時のバックアップボイラーの年間平均効率 (-) (6)
def get_e_BB_ave(L_BB_DHW_d_t, L_BB_DHW_ba2_d_t, E_G_BB_DHW_d_t, E_G_BB_DHW_ba2_d_t):
    """
    :param L_BB_DHW_d_t: 1時間当たりのバックアップボイラーが分担する給湯熱負荷 (MJ/h)
    :param L_BB_DHW_ba2_d_t: 1時間当たりの浴槽追焚時におけるバックアップボイラーが分担する給湯熱負荷 (MJ/h)
    :param E_G_BB_DHW_d_t: 1時間当たりの給湯時のバックアップボイラーのガス消費量 (MJ/h)
    :param E_G_BB_DHW_ba2_d_t: 1時間当たりの浴槽追焚時におけるバックアップボイラーのガス消費量 (MJ/d)
    :return: 給湯時のバックアップボイラーの年間平均効率 (-)
    """
    e_BB_ave = np.sum(L_BB_DHW_d_t - L_BB_DHW_ba2_d_t) / np.sum(E_G_BB_DHW_d_t - E_G_BB_DHW_ba2_d_t)
    return e_BB_ave


### 12. 給湯時のバックアップボイラーのガス消費量

#### 12.1 ガス消費量

日付$d$の時刻$t$における1時間当たりの給湯時のバックアップボイラーのガス消費量$E_{G,BB,DHW,d,t}$および浴槽追焚時におけるバックアップボイラーのガス消費量$E_{G,BB,DHW,ba2,d,t}$は、ふろ機能の種類、日付$d$の時刻$t$における1時間当たりの台所水栓、浴室シャワー水栓、洗面水栓、浴槽水栓湯はり、浴槽自動湯はり、浴槽水栓さ
し湯、及び浴槽追焚時におけるバックアップボイラーが分担する給湯熱負荷$L_{BB,DHW,k,d,t}$、$L_{BB,DHW,s,d,t}$、$L_{BB,DHW,w,d,t}$、$L_{BB,DHW,b1,d,t}$、$L_{BB,DHW,b2,d,t}$、$L_{BB,DHW,ba1,d,t}$及び$L_{BB,DHW,ba2,d,t}$、日付$d$における日平均外気温度$\theta_{ex,ave,d}$に応じて、付録 D により求まる。 

#### 12.2 バックアップボイラーが分担する給湯熱負荷

$$
\displaystyle
L_{BB,DHW,d,t} = L_{BB,DHW,k,d,t}+L_{BB,DHW,s,d,t}+L_{BB,DHW,w,d,t}+L_{BB,DHW,b1,d,t}+L_{BB,DHW,b2,d,t}+L_{BB,DHW,ba1,d,t}+L_{BB,DHW,ba2,d,t}
$$

$$
\displaystyle
L_{BB,DHW,k,d,t} = L''_{k,d,t} - Q_{gen,DHW,d} \times \frac{L''_{k,d,t}}{L_{DHW,d}}
$$

$$
\displaystyle
L_{BB,DHW,s,d,t} = L''_{s,d,t} - Q_{gen,DHW,d} \times \frac{L''_{s,d,t}}{L_{DHW,d}}
$$

$$
\displaystyle
L_{BB,DHW,w,d,t} = L''_{w,d,t} - Q_{gen,DHW,d} \times \frac{L''_{w,d,t}}{L_{DHW,d}}
$$

$$
\displaystyle
L_{BB,DHW,b1,d,t} = L''_{b1,d,t} - Q_{gen,DHW,d} \times \frac{L''_{b1,d,t}}{L_{DHW,d}}
$$

$$
\displaystyle
L_{BB,DHW,b2,d,t} = L''_{b2,d,t} - Q_{gen,DHW,d} \times \frac{L''_{b2,d,t}}{L_{DHW,d}}
$$

$$
\displaystyle
L_{BB,DHW,ba1,d,t} = L''_{ba1,d,t} - Q_{gen,DHW,d} \times \frac{L''_{ba1,d,t}}{L_{DHW,d}}
$$

$$
\displaystyle
L_{BB,DHW,ba2,d,t} = L''_{ba2,d,t}
$$

$L_{BB,DHW,k,d,t}$：日付$d$の時刻$t$における1時間当たりの台所水栓におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,s,d,t}$：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,w,d,t}$：日付$d$の時刻$t$における1時間当たりの洗面水栓におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,b1,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はりにおけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,b2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はりにおけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,ba1,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{BB,DHW,ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$Q_{gen,DHW,d}$：日付$d$における1日当たりの給湯の排熱利用量(MJ/d)  
$L''_{k,d,t}$：日付$d$の時刻$t$における1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h)  
$L''_{w,d,t}$：日付$d$の時刻$t$における1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h)  
$L''_{s,d,t}$：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h)  
$L''_{b1,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はりにおける太陽熱補正給湯熱負荷(MJ/h)  
$L''_{b2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はりにおける太陽熱補正給湯熱負荷(MJ/h)  
$L''_{ba1,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷(MJ/h)  
$L''_{ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚における太陽熱補正給湯熱負荷(MJ/h)  

In [ ]:
# 1時間あたりのバックアップボイラーが分担する給湯熱負荷 (7)
def get_L_BB_x_d_t(L_dashdash_k_d_t, L_dashdash_w_d_t, L_dashdash_s_d_t,
                   L_dashdash_b1_d_t, L_dashdash_b2_d_t, L_dashdash_ba1_d_t, L_dashdash_ba2_d_t,
                   Q_gen_DHW_d, L_DHW_d):
    """
    :param L_BB_DHW_k_d_t: 1時間当たりの台所水栓における給湯のバックアップボイラーが分担する給湯熱負荷 (MJ/h)
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はりにおける太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動湯はりにおける太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷 (MJ/h)
    :param Q_gen_DHW_d: 1日当たりの給湯の排熱利用量 (MJ/d)
    :param L_DHW_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :return:
    """

    Q_gen_DHW_d = np.repeat(Q_gen_DHW_d, 24)
    L_DHW_d = np.repeat(L_DHW_d, 24)

    # (7b)
    L_BB_DHW_k_d_t = L_dashdash_k_d_t - Q_gen_DHW_d * (L_dashdash_k_d_t / L_DHW_d)

    # (7c)
    L_BB_DHW_s_d_t = L_dashdash_s_d_t - Q_gen_DHW_d * (L_dashdash_s_d_t / L_DHW_d)

    # (7d)
    L_BB_DHW_w_d_t = L_dashdash_w_d_t - Q_gen_DHW_d * (L_dashdash_w_d_t / L_DHW_d)

    # (7e)
    L_BB_DHW_b1_d_t = L_dashdash_b1_d_t - Q_gen_DHW_d * (L_dashdash_b1_d_t / L_DHW_d)

    # (7f)
    L_BB_DHW_b2_d_t = L_dashdash_b2_d_t - Q_gen_DHW_d * (L_dashdash_b2_d_t / L_DHW_d)

    # (7g)
    L_BB_DHW_ba1_d_t = L_dashdash_ba1_d_t - Q_gen_DHW_d * (L_dashdash_ba1_d_t / L_DHW_d)

    # (7h)
    L_BB_DHW_ba2_d_t = L_dashdash_ba2_d_t

    # (7a)
    L_BB_DHW_d_t = L_BB_DHW_k_d_t + \
                   L_BB_DHW_s_d_t + \
                   L_BB_DHW_w_d_t + \
                   L_BB_DHW_b1_d_t + \
                   L_BB_DHW_b2_d_t + \
                   L_BB_DHW_ba1_d_t + \
                   L_BB_DHW_ba2_d_t

    return L_BB_DHW_d_t, L_BB_DHW_k_d_t, L_BB_DHW_s_d_t, L_BB_DHW_w_d_t, \
           L_BB_DHW_b1_d_t, L_BB_DHW_b2_d_t, L_BB_DHW_ba1_d_t, L_BB_DHW_ba2_d_t

### 13. 温水暖房時のバックアップボイラーのガス消費量

日付$d$の時刻$t$における1 時間当たりの温水暖房時のバックアップボイラーのガス消費量$E_{G,BB,HWH,d,t}$は、温水暖房のバックアップボイラーの種類、定格効率、定格能力、日付$d$の時刻$t$における 1 時間当たりのバックアップボイラーが分担する温水暖房熱負荷$L_{BB,HWH,d,t}$及び日付$d$の時刻$t$における 1 時間平均の温水暖房用熱源機の往き温水温度$\theta_{SW,HWH,d,t}$に依存し、付録 E によるものとする。

$$\large
\displaystyle
L_{BB,HWH,d,t} = \left\{ \begin{array}{ll}
    L_{HWH,d,t} - Q_{gen,HWH,d} \times \frac{L_{HWH,d,t}}{L_{HWH,d}} & (L_{HWH,d} > Q_{gen,HWH,d})\\ 
    0 & (L_{HWH,d} \leq Q_{gen,HWH,d})\\ 
    \end{array}
\right.
$$

$L_{HWH,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房の熱負荷(MJ/h)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  
$Q_{gen,HWH,d}$：日付$d$における1日当たりの温水暖房の排熱利用量(MJ/d)  

In [ ]:
# 1時間当たりのバックアップボイラーが分担する温水暖房熱負荷 (MJ/h) (8)
def get_L_BB_HWH_d_t(L_HWH_d_t, L_HWH_d, Q_gen_HWH_d):
    """
    :param L_HWH_d_t: 1時間当たりの温水暖房の熱負荷 (MJ/h)
    :param L_HWH_d: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :param Q_gen_HWH_d: 1日当たりの温水暖房の排熱利用量 (MJ/h)
    :return: 1時間当たりのバックアップボイラーが分担する温水暖房熱負荷 (MJ/h)
    """
    L_BB_HWH_d_t = np.zeros(24 * 365)

    L_HWH_d = np.repeat(L_HWH_d, 24)
    Q_gen_HWH_d = np.repeat(Q_gen_HWH_d, 24)

    # L_HWH_d ≠ 0 の場合
    f1 = (L_HWH_d > Q_gen_HWH_d)
    L_BB_HWH_d_t[f1] = L_HWH_d_t[f1] - Q_gen_HWH_d[f1] * (L_HWH_d_t[f1] / L_HWH_d[f1])

    # L_HWH_d = 0 の場合
    f2 = (L_HWH_d <= Q_gen_HWH_d)
    L_BB_HWH_d_t[f2] = 0

    return L_BB_HWH_d_t

### 14. 発電ユニット

#### 14.1 排熱利用量 

&#9312;温水暖房への排熱利用がない場合

$$\large
\displaystyle
Q_{gen,DHW,d} = \min \left( Q_{PU,gen,d} \times r_{DHW,gen,PU},L_{DHW,d} \right)
$$

$$\large
\displaystyle
Q_{gen,HWH,d} = 0.0
$$

&#9313;温水暖房への排熱利用がある場合で給湯優先の機種

$$\large
\displaystyle
Q_{gen,DHW,d} = \min \left( Q_{PU,gen,d} \times r_{DHW,gen,PU},L_{DHW,d} \right)
$$

$$\large
\displaystyle
Q_{gen,HWH,d} = \min \left( (Q_{PU,gen,d} - Q_{gen,DHW,d}) \times r_{HWH,gen,PU},L_{HWH,d} \right)
$$

&#9314;温水暖房への排熱利用がある場合で温水暖房優先の機種

$$\large
\displaystyle
Q_{gen,HWH,d} = \min \left( Q_{PU,gen,d} \times r_{HWH,gen,PU},L_{HWH,d} \right)
$$

$$\large
\displaystyle
Q_{gen,DHW,d} = \min \left( (Q_{PU,gen,d} - Q_{gen,HWH,d}) \times r_{DHW,gen,PU},L_{DHW,d} \right)
$$

$Q_{gen,DHW,d}$：日付$d$における1日当たりの給湯の排熱利用量(MJ/d)  
$Q_{gen,HWH,d}$：日付$d$における1日当たりの温水暖房の排熱利用量(MJ/d)  
$Q_{PU,gen,d}$：日付$d$における1日当たりの発電ユニット排熱量(MJ/d)  
$r_{DHW,gen,PU}$：発電ユニットの給湯排熱利用率  
$r_{HWH,gen,PU}$：発電ユニットの温水暖房排熱利用率  
$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  

In [ ]:
# 1日当たりの給湯・温水暖房の排熱利用量 (MJ/d) (9)(10)(11)
def calc_Q_gen_x_d(exhaust, exhaust_priority, Q_PU_gen_d, r_DHW_gen_PU_d, r_HWH_gen_PU_d, L_DHW_d, L_HWH_d):
    """
    :param exhaust: 温水暖房への排熱利用がある場合はTrue
    :param exhaust_priority: 温水暖房への排熱利用がある場合の排熱の優先先
    :param Q_PU_gen_d:　1日当たりの発電ユニット排熱量 (MJ/d)
    :param r_HWH_gen_PU_d: 発電ユニットの温水暖房排熱利用率 (-)
    :param L_DHW_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :return: 1日当たりの給湯・温水暖房の排熱利用量をそれぞれ (MJ/d)
    """
    if exhaust == False:
        # (9b)
        Q_gen_HWH_d = np.zeros(365)
        # (9a)
        Q_gen_DHW_d = np.clip(Q_PU_gen_d * r_DHW_gen_PU_d, None, L_DHW_d)
    elif exhaust_priority == '給湯優先':
        # (10a)
        Q_gen_DHW_d = np.clip(Q_PU_gen_d * r_DHW_gen_PU_d, None, L_DHW_d)
        # (10b)
        Q_gen_HWH_d = np.clip((Q_PU_gen_d - Q_gen_DHW_d) * r_HWH_gen_PU_d, None, L_HWH_d)
    elif exhaust_priority == '温水暖房優先':
        # (11a)
        Q_gen_HWH_d = np.clip(Q_PU_gen_d * r_HWH_gen_PU_d, None, L_HWH_d)
        # (11b)
        Q_gen_DHW_d = np.clip((Q_PU_gen_d - Q_gen_HWH_d) * r_HWH_gen_PU_d, None, L_DHW_d)
    else:
        raise ValueError((exhaust, exhaust_priority))

    return Q_gen_DHW_d, Q_gen_HWH_d

$$\large
\displaystyle
Q_{PU,gen,d} = E_{G,PU,d} \times e_{H,PU,d}
$$

$E_{G,PU,d}$：日付$d$における1日当たりの発電ユニットのガス消費量(MJ/d)  
$e_{H,PU,d}$：日付$d$における発電ユニットの日平均排熱効率  

In [ ]:
# 1日当たりの発電ユニット排熱量 (12)
def get_Q_PU_gen_d(E_G_PU_d, e_H_PU_d):
    """
    :param E_G_PU_d: 1日当たりの発電ユニットのガス消費量 (MJ/d)
    :param e_H_PU_d: 発電ユニットの日平均排熱効率 (-)
    :return: 1日当たりの発電ユニット排熱量 (MJ/d)
    """
    return E_G_PU_d * e_H_PU_d

#### 14.2 発電量

$$\large
E_{E,gen,PU,d,t} = E_{G,PU,d,t} \times e_{E,PU,d} \div 3.6
$$

$E_{E,gen,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットの発電量(kWh/h)  
$E_{G,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットのガス消費量(MJ/h)  
$e_{E,PU,d}$：日付$d$における発電ユニットの日平均発電効率  

In [ ]:
# 1時間当たりの発電ユニットの発電量 (kWh/h) (13)
def get_E_E_gen_PU_d_t(E_G_PU_d_t, e_E_PU_d):
    """
    :param E_G_PU_d_t: 1時間当たりの発電ユニットのガス消費量 (MJ/h)
    :param e_E_PU_d: 発電ユニットの日平均発電効率 (-)
    :return: 発電ユニットの発電量 (kWh/h)
    """
    e_E_PU_d = np.repeat(e_E_PU_d, 24)

    E_E_gen_PU_d_t =  E_G_PU_d_t * e_E_PU_d / 3.6

    return E_E_gen_PU_d_t

#### 14.3 ガス消費量

$t^*_{PU,start,d}< 23$ の場合

$$\large
\displaystyle
E_{G,PU,d,t} = \left\{ \begin{array}{ll}
    0 & (t < t^*_{PU,start,d})\\ 
    E_{G,PU,d} - E'_{G,PU,d,t+1} & (t = t^*_{PU,start,d})\\ 
    E_{E,PU,d,t} \times 3.6 \div e_{E,PU,d} & (t > t^*_{PU,start,d})\\ 
    \end{array}
\right.
$$

$t^*_{PU,start,d}= 23$ の場合

$$\large
\displaystyle
E_{G,PU,d,t} = \left\{ \begin{array}{ll}
    0 & (t < t^*_{PU,start,d})\\ 
    E_{G,PU,d} & (t = t^*_{PU,start,d})\\ 
    \end{array}
\right.
$$

$E_{E,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  
$e_{E,PU,d}$：日付$d$における発電ユニットの日平均発電効率(-)  
$E_{G,PU,d}$：日付$d$における1日当たりの発電ユニットのガス消費量(MJ/d)  
$E'_{G,PU,d,t+1}$：日付$d$の時刻$t+1$における当該時刻から 23 時までの発電ユニットのガス消費量(MJ)  
$t^*_{PU,start,d}$：日付$d$における発電ユニットの稼働開始時刻(-)  

In [ ]:
# 1時間当たりの発電ユニットのガス消費量 (MJ/h) (14a)
def calc_E_G_PU_d_t(E_G_PU_d, E_E_PU_d, E_E_PU_d_t, e_E_PU_d):
    """
    :param E_G_PU_d: 1日当たりの発電ユニットのガス消費量 (MJ/d)
    :param E_E_PU_d: 1日当たりの発電ユニットの分担可能電力力負荷 (kWh/d)
    :param E_E_PU_d_t: 1時間当たりの発電ユニットの分担可能電力力負荷 (kWh/h)
    :param e_E_PU_d: 発電ユニットの日平均発電効率 (-)
    :return: 1時間当たりの発電ユニットのガス消費量 (MJ/h)
    """

    # 日付dの時刻hにおける当該時刻から23時までの発電ユニットのガス消費量 (MJ/d) (14c)
    E_dash_G_PU_d_h = get_E_dash_G_PU_d_h(E_E_PU_d_t, e_E_PU_d)

    # 日付dにおける発電ユニットの稼働開始時刻 (h) (14b)
    t_star_PU_start_d = get_t_star_PU_start_d(E_G_PU_d, E_dash_G_PU_d_h)

    # 時刻tの表を作成
    t = np.tile(np.arange(24), 365)

    # 計算結果格納領域
    E_G_PU_d_t = np.zeros(24 * 365)

    # 計算用に24時間化
    E_G_PU_d = np.repeat(E_G_PU_d, 24)
    e_E_PU_d = np.repeat(e_E_PU_d, 24)
    t_star_PU_start_d = np.repeat(t_star_PU_start_d, 24)

    # E_dash_G_PU_d_h の1時間後の配列の作成
    E_dash_G_PU_d_h_plus_1h = np.roll(E_dash_G_PU_d_h, -1)

    # 条件1: t_star_PU_start_d < 23 and t < t_star_PU_start_d
    f1 = np.logical_and(t_star_PU_start_d < 23, t < t_star_PU_start_d)
    E_G_PU_d_t[f1] = 0

    # 条件2: t_star_PU_start_d < 23 and t = t_star_PU_start_d
    f2 = np.logical_and(t_star_PU_start_d < 23, t == t_star_PU_start_d)
    E_G_PU_d_t[f2] = E_G_PU_d[f2] - E_dash_G_PU_d_h_plus_1h[f2]

    # 条件3: t_star_PU_start_d < 23 and t > t_star_PU_start_d
    f3 = np.logical_and(t_star_PU_start_d < 23, t > t_star_PU_start_d)
    E_G_PU_d_t[f3] = E_E_PU_d_t[f3] * 3.6 / e_E_PU_d[f3]

    # 条件4: t_star_PU_start_d = 23 and t < t_star_PU_start_d
    f4 = np.logical_and(t_star_PU_start_d == 23, t < t_star_PU_start_d)
    E_G_PU_d_t[f4] = 0

    # 条件5: t_star_PU_start_d = 23 and t = t_star_PU_start_d
    f5 = np.logical_and(t_star_PU_start_d == 23, t == t_star_PU_start_d)
    E_G_PU_d_t[f5] = E_G_PU_d[f5]

    return E_G_PU_d_t

$$\large
\displaystyle
t^*_{PU,start,d} = max \{ h| E_{G,PU,d} \leq E'_{G,PU,d,h} \}
$$

$E'_{G,PU,d,h}$：日付$d$の時刻$h$における当該時刻から 23 時までの発電ユニットのガス消費量(MJ)  
$E_{G,PU,d}$：日付$d$における1日当たりの発電ユニットのガス消費量(MJ/d)  

In [ ]:
# 日付dにおける発電ユニットの稼働開始時刻 (h) (14b)
def get_t_star_PU_start_d(E_G_PU_d, E_dash_G_PU_d_h):
    """
    :param E_G_PU_d: # 1日当たりの発電ユニットのガス消費量 (MJ/d)
    :param E_dash_G_PU_d_h: # 日付dの時刻hにおける当該時刻から23時までの発電ユニットのガス消費量 (MJ/d)
    :return: 日付dにおける発電ユニットの稼働開始時刻 (h)
    """
    # 0,1,2,3...23, 0,1,2...23, .. と 24*365のインデックスを作成
    index_map = np.tile(np.arange(24), 365)

    # E_G_PU_d <= E_dash_PU_d_h を満たすインデックスをTrueにする
    bool_map = np.repeat(E_G_PU_d, 24) <= E_dash_G_PU_d_h

    # index_map のうち、 bool_mapがFalseになっている箇所を0にする
    index_map = index_map * bool_map

    # index_map を 365 * 24 の2次元配列にする
    index_map = np.reshape(index_map, (365, 24))

    # 1日単位で最大のインデックスを取得
    t_star_PU_start_d = np.max(index_map, axis=1)

    return t_star_PU_start_d

$$\large
\displaystyle
E'_{G,PU,d,h} = \sum^{23}_{t=h} E_{E,PU,d,t} \times 3.6 \div e_{E,PU,d}
$$

$E_{E,PU,d,t}$：日付$d$の時刻$h$における 1 時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  
$e_{E,PU,d}$：日付$d$における発電ユニットの日平均発電効率(-)  

In [ ]:
# 日付dの時刻hにおける当該時刻から23時までの発電ユニットのガス消費量 (MJ/d) (14c)
def get_E_dash_G_PU_d_h(E_E_PU_d_t, e_E_PU_d):
    """
    :param E_E_PU_d_t: 日付dの時刻tにおける1時間当たりの発電ユニットの分担可能電力負荷 (kWh/h)
    :param e_E_PU_d: 日付dにおける発電ユニットの日平均発電効率 (-)
    :return: 日付dの時刻hにおける当該時刻から23時までの発電ユニットのガス消費量 (MJ/d)
    """
    E_E_PU_d_t = np.reshape(E_E_PU_d_t, (365, 24))

    E_dash_G_PU_d_h = np.zeros((365, 24))
    for h in range(24):
        E_dash_G_PU_d_h[:, h] = np.sum(E_E_PU_d_t[:, h:24], axis=1) * 3.6 / e_E_PU_d

    return np.reshape(E_dash_G_PU_d_h, 24*365)

&#9312;発電ユニットの発電方式が「熱主」の場合 

$$\large
E_{G,PU,d} = min(E_{G,PU,EVt,d}, E_{G,PU,HVt,d})
$$

&#9313;発電ユニットの発電方式が「電主」の場合

$$\large
E_{G,PU,d} = E_{G,PU,EVt,d}
$$

$E_{G,PU,EVt,d}$：日付$d$における1日当たりの発電ユニットの発電量推定時の仮想ガス消費量(MJ/d)  
$E_{G,PU,HVt,d}$：日付$d$における1日当たりの発電ユニットの排熱量推定時の仮想ガス消費量(MJ/d)  

In [ ]:
# 1日当たりの発電ユニットのガス消費量 (MJ/d) (15)
def get_E_G_PU_d(PU_type, E_G_PU_EVt_d, E_G_PU_HVt_d=None):
    """
    :param PU_type: 発電ユニットの発電方式
    :param E_G_PU_EVt_d: 1日当たりの発電ユニットの発電量推定時の仮想ガス消費量 (MJ/d)
    :param E_G_PU_HVt_d: 1日当たりの発電ユニットの排熱量推定時の仮想ガス消費量 (MJ/d)
    :return: 1日当たりの発電ユニットの燃料消費量 (MJ/d)
    """
    if PU_type == '熱主':
        # (15a)
        return np.clip(E_G_PU_EVt_d, None, E_G_PU_HVt_d)
    elif PU_type == '電主':
        # (15b)
        return E_G_PU_EVt_d
    else:
        raise ValueError(PU_type)


$$\large
E_{G,PU,EVt,d} = E_{E,gen,PU,EVt,d} \times 3.6 \div e_{E,PU,d} 
$$

$E_{E,gen,PU,EVt,d}$：日付$d$における1日当たりの発電ユニットの発電量推定時の仮想発電量(kWh/d)  
$e_{E,PU,d}$：日付$d$における発電ユニットの日平均発電効率  

In [ ]:
# 1日当たりの発電ユニットの発電量推定時の仮想ガス消費量 (MJ/d) (16)
def get_E_G_PU_EVt_d(E_E_gen_PU_EVt_d, e_E_PU_d):
    """
    :param E_E_gen_PU_EVt_d: 1日当たりの発電ユニットの発電量推定時の仮想発電量 (kWh/d)
    :param e_E_PU_d: 発電ユニットの日平均発電効率 (-)
    :return: 1日当たりの発電ユニットの発電量推定時の仮想燃料消費量 (MJ/d)
    """
    return E_E_gen_PU_EVt_d * 3.6 / e_E_PU_d


$$\large
\displaystyle
E_{E,gen,PU,EVt,d} = \min ( a_{PU} \times E_{E,PU,d} \times 3.6 + a_{DHW} \times L_{DHW,d} + a_{HWH} \times L_{HWH,d} \\
\large+ b, E_{E,PU,d} \times c \times 3.6) \div 3.6
$$

$E_{E,PU,d}$：日付$d$における1日当たりの発電ユニットの分担可能電力負荷(kWh/d)  
$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  

In [ ]:
# 1日当たりの発電ユニットの発電量推定時の仮想発電量 (kWh/d) (17)
def get_E_E_gen_PU_EVt_d(E_E_PU_d, L_DHW_d, L_HWH_d, a_PU, a_DHW, a_HWH, b, c):
    """
    :param CG_category: コージェネレーション設備の種類
    :param E_E_PU_d: 1日当たりの発電ユニットの分担可能電力負荷 (kWh/d)
    :param L_DHW_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH_d: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :return: 1日当たりの発電ユニットの発電量推定時の仮想発電量
    """
    E_E_gen_PU_EVt_d = np.clip(a_PU * E_E_PU_d * 3.6 + a_DHW * L_DHW_d + a_HWH * L_HWH_d + b, None,
                               E_E_PU_d * c * 3.6) / 3.6
    return E_E_gen_PU_EVt_d

$$\large
\displaystyle
E_{G,PU,HVt,d} = (a_{DHW} \times L_{DHW,d} + a_{HWH} \times L_{HWH,d}) \times r_{H,gen,PU,HVt,d} \div e_{H,PU,d}
$$

$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  
$r_{H,gen,PU,HVt,d}$：日付$d$における発電ユニットの排熱量推定時の仮想排熱量上限比  
$e_{H,PU,d}$：日付$d$における発電ユニットの日平均排熱効率  

In [ ]:
# 1日当たりの発電ユニットの排熱量推定時の仮想燃料消費 (MJ/d) (18)
def get_E_G_PU_HVt_d(e_H_PU_d, L_DHW_d, L_HWH_d, r_H_gen_PU_HVt_d, a_DHW, a_HWH):
    """
    :param e_H_PU_d: 発電ユニットの日平均排熱効率 (-)
    :param L_DHW_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH_d: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :param r_H_gen_PU_HVt_d: 発電ユニットの排熱量推定時の仮想排熱量上限比 (-)
    :param a_DHW: パラメータ a_DHW
    :param a_HWH: パラメータ a_HWH
    :return: 1日当たりの発電ユニットの排熱量推定時の仮想燃料消費 (MJ/d)
    """
    E_G_PU_HVt_d = (a_DHW * L_DHW_d + a_HWH * L_HWH_d) * r_H_gen_PU_HVt_d / e_H_PU_d
    return E_G_PU_HVt_d

$$\large
\displaystyle
r_{H,gen,PU,HVt,d} = a_{DHW} \times L_{DHW,d} + a_{HWH} \times L_{HWH,d} + b
$$

$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  

In [ ]:
# 発電ユニットの排熱量推定時の仮想排熱量上限比 (-) (19)
def get_r_H_gen_PU_HVt_d(L_DHW_d, L_HWH_d, a_DHW, a_HWH, b):
    """
    :param L_DHW_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH_d: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :param a_DHW: パラメータ a_DHW
    :param a_HWH: パラメータ a_HWH
    :param b: パラメータ b
    :return: 発電ユニットの排熱量推定時の仮想排熱量上限比
    """
    r_H_gen_PU_HVt_d = a_DHW * L_DHW_d + a_HWH * L_HWH_d + b
    return r_H_gen_PU_HVt_d

#### 14.4 発電効率 

発電ユニットの発電効率の上限値を上回る場合は上限値に等しいとし、下限値を下回る場合は下限値に等しいとする。

$$\large
\displaystyle
e_{E,PU,d} = a_{PU} \times E_{E,PU,d} \times 3.6 + a_{DHW} \times L_{DHW,d} + a_{HWH} \times L_{HWH,d} + b
$$

$E_{E,PU,d}$：日付$d$における1日当たりの発電ユニットの分担可能電力負荷(kWh/d)  
$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  

In [ ]:
# 発電ユニットの日平均発電効率 (-) (20)
def get_e_E_PU_d(E_E_PU_d, L_DHW_d, L_HWH_d, a_PU, a_DHW, a_HWH, b, e_E_PU_d_max, e_E_PU_d_min):
    """
    :param E_E_PU_d: 1日当たりの発電ユニットの分担可能電力負荷 (kWh/d)
    :param L_DHW_d: 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH_d: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :param a_PU: パレメータ a_PU
    :param a_DHW: パラメータ a_DHW
    :param a_HWH: パラメータ a_HWH
    :param b: パラメータ b
    :param e_E_PU_d_max: 上限値
    :param e_E_PU_d_min: 下限値
    :return: 発電ユニットの日平均発電効率 (-)
    """

    e_E_PU_d = a_PU * E_E_PU_d * 3.6 + a_DHW * L_DHW_d + a_HWH * L_HWH_d + b

    return np.clip(e_E_PU_d, e_E_PU_d_min, e_E_PU_d_max)

#### 14.5 排熱効率 

発電ユニットの排熱効率の上限値を上回る場合は上限値に等しいとし、下限値を下回る場合は下限値に等しいとする。

$$\large
\displaystyle
e_{H,PU,d} = a_{PU} \times E_{E,PU,d} \times 3.6 + a_{DHW} \times L_{DHW,d} + a_{HWH} \times L_{HWH,d} + b
$$

$E_{E,PU,d}$：日付$d$における1日当たりの発電ユニットの分担可能電力負荷(kWh/d)  
$L_{DHW,d}$：日付$d$における1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/d)  
$L_{HWH,d}$：日付$d$における1日当たりの温水暖房の熱負荷(MJ/d)  

In [ ]:
# 発電ユニットの日平均排熱効率 (-) (21)
def get_e_H_PU_d(E_E_PU_d, L_DHW_d, L_HWH_d, a_PU, a_DHW, a_HWH, b, e_H_PU_d_min, e_H_PU_d_max):
    """
    :param E_E_PU_d: 1日当たりの発電ユニットの分担可能電力負荷 (kWh/d)
    :param L_DHW_d: 1日当たりの浴槽追焚をの除く太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH_d: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :param a_PU: パレメータ a_PU
    :param a_DHW: パラメータ a_DHW
    :param a_HWH: パラメータ a_HWH
    :param b: パラメータ b
    :param e_H_PU_d_max: 上限値
    :param e_H_PU_d_min: 下限値
    :return: 発電ユニットの日平均排熱効率 (-)
    """
    e_H_PU_d = a_PU * E_E_PU_d * 3.6 + a_DHW * L_DHW_d + a_HWH * L_HWH_d + b

    return np.clip(e_H_PU_d, e_H_PU_d_min, e_H_PU_d_max)


#### 14.6 分担可能電力負荷 

$$\large
\displaystyle
E_{E,PU,d} = \sum E_{E,PU,d,t}
$$

$E_{E,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  

In [ ]:
# 1日当たりの発電ユニットの分担可能電力負荷 (kWh/d) (22)
def get_E_E_PU_d(E_E_PU_d_t):
    """
    :param E_E_PU_d_t: 1時間当たりの発電ユニットの分担可能電力負荷 (kWh/h)
    :return: 1日当たりの発電ユニットの分担可能電力負荷 (kWh/d)
    """
    tmp = E_E_PU_d_t.reshape((365, 24))
    E_E_PU_d = np.sum(tmp, axis=1)

    return E_E_PU_d

$$\large
\displaystyle
E_{E,PU,d,t} = \left\{ \begin{array}{ll}
    \min \left( E_{E,dmd,PU,d,t}, P_{rtd,PU} \times 10^{-3} \right) & (逆潮流がない場合)\\ 
    P_{rtd,PU} \times 10^{-3} & (逆潮流がある場合)\\ 
    \end{array}
\right.
$$

$E_{E,dmd,PU,d,t}$：日付$d$の時刻$t$における1時間当たりの発電ユニットの電力需要(kWh/h)  
$P_{rtd,PU}$：定格発電出力(W)  

In [ ]:
# 1時間当たりの発電ユニットの分担可能電力負荷 (kWh/h) (23)
def get_E_E_PU_d_t(E_E_dmd_PU_d_t, P_rtd_PU, reverse):
    """
    :param E_E_dmd_PU_d_t: 1時間当たりの発電ユニットの電力需要 (kWh/h)
    :param P_rtd_PU: 定格発電出力 (W)
    :return: 1時間当たりの発電ユニットの分担可能電力負荷  (kWh/h)
    """
    if reverse == False:
        return np.clip(E_E_dmd_PU_d_t, None, P_rtd_PU * 10 ** (-3))
    else:
        return np.ones_like(E_E_dmd_PU_d_t) * P_rtd_PU * 10 ** (-3)


$$\large
\displaystyle
E_{E,dmd,PU,d,t} = E_{E,dmd,d,t} + E_{E,TU,aux,d,t}
$$

$E_{E,dmd,d,t}$：日付$d$の時刻$t$における1時間当たりの電力需要(kWh/h)  
$E_{E,TU,aux,d,t}$：日付$d$の時刻$t$における1時間当たりのタンクユニットの補機消費電力量(kWh/h)  

In [ ]:
# 発電ユニットの電力需要 (kWh/h) (24)
def get_E_E_dmd_PU_d_t(E_E_dmd_d_t, E_E_TU_aux_d_t):
    """
    :param E_E_dmd_d_t: 1時間当たりの電力需要 (kWh/h)
    :param E_E_TU_aux_d_t: 1時間当たりのタンクユニットの補機消費電力量 (kWh/h)
    :return: 発電ユニットの電力需要 (kWh/h)
    """
    E_E_dmd_PU_d_t = E_E_dmd_d_t + E_E_TU_aux_d_t

    return E_E_dmd_PU_d_t

### 15. タンクユニットの補機消費電力

$$\large
\displaystyle
E_{E,TU,aux,d,t} = E_{E,TU,aux,DHW,d,t} + E_{E,TU,aux,HWH,d,t} + E_{E,TU,aux,ba2,d,t}
$$

$E_{E,TU,aux,DHW,d,t}$：日付$d$の時刻$t$における1時間当たりの給湯時のタンクユニットの補機消費電力量(kWh/h)  
$E_{E,TU,aux,HWH,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房時のタンクユニットの補機消費電力量(kWh/h)  
$E_{E,TU,aux,ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚のタンクユニットの補機消費電力量(kWh/h)  

In [ ]:
# 1時間当たりのタンクユニットの補機消費電力量 (25)
def get_E_E_TU_aux_d_t(E_E_TU_aux_DHW_d_t, E_E_TU_aux_HWH_d_t, E_E_TU_aux_ba2_d_t):
    """
    :param E_E_TU_aux_DHW_d_t: 1時間当たりの給湯時のタンクユニットの補機消費電力量 (kWh/h)
    :param E_E_TU_aux_HWH_d_t: 1時間当たりの温水暖房時のタンクユニットの補機消費電力量 (kWh/h)
    :param E_E_TU_aux_ba2_d_t: 1時間当たりの浴槽追焚のタンクユニットの補機消費電力量 (kWh/h)
    :return: 1時間当たりのタンクユニットの補機消費電力量 (kWh/h)
    """
    E_E_TU_aux_d_t = E_E_TU_aux_DHW_d_t + E_E_TU_aux_HWH_d_t + E_E_TU_aux_ba2_d_t
    return E_E_TU_aux_d_t

$$\large
\displaystyle
E_{E,TU,aux,DHW,d,t} = P_{TU,aux,DHW} \times 10^{-3}
$$

$P_{TU,aux,DHW}$：給湯のタンクユニットの補機消費電力(W)  

In [ ]:
# 1時間当たりの給湯時のタンクユニットの補機消費電力量 (kWh/h) (26)
def get_E_E_TU_aux_DHW_d_t(P_TU_aux_DHW):
    """
    :param P_TU_aux_DHW: 給湯のタンクユニットの補機消費電力 (W)
    :return: 1時間当たりの給湯時のタンクユニットの補機消費電力量 (kWh/h)
    """
    E_E_TU_aux_DHW_d_t = P_TU_aux_DHW * 10 ** (-3)
    return E_E_TU_aux_DHW_d_t

$$\large
\displaystyle
E_{E,TU,aux,HWH,d,t} = P_{TU,aux,HWH} \times r_{WS,HWH,d,t} \times 10^{-3}
$$

$P_{TU,aux,HWH}$：温水暖房のタンクユニットの補機消費電力(W)  
$r_{WS,HWH,d,t}$：日付$d$の時刻$t$における温水暖房の温水供給運転率 

温水暖房のタンクユニットの補機消費電力$P_{TU,aux,HWH}$は、温水暖房への排熱利用がある場合は付録A に定める値とし、温水暖房への排熱利用がない場合は 73W とする。 

In [ ]:
# 1時間当たりの温水暖房時のタンクユニットの補機消費電力量 (kWh/h) (27)
def get_E_E_TU_aux_HWH_d_t(exhaust, P_TU_aux_HWH=None, r_WS_HWH_d_t=None):
    """
    :param exhaust: 温水暖房への排熱利用がある場合はTrue
    :param P_TU_aux_HWH: 給湯のタンクユニットの補機消費電力 (W)
    :param r_WS_HWH_d_t: 温水暖房の温水供給運転率 (-)
    :return: 1時間当たりの給湯時のタンクユニットの補機消費電力量 (kWh/h)
    """
    if exhaust:
        # -- ①温水暖房への排熱利用がある場合 --
        E_E_TU_aux_HWH_d_t = P_TU_aux_HWH * r_WS_HWH_d_t * 10 ** (-3)
    else:
        # -- ②温水暖房への排熱利用がない場合 --
        E_E_TU_aux_HWH_d_t = 73.0 * r_WS_HWH_d_t * 10 ** (-3)

    return E_E_TU_aux_HWH_d_t

$$\large
\displaystyle
L_{BB,DHW,ba2,d,t} = L''_{ba2,d,t}
$$

$L_{BB,DHW,ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚時におけるバックアップボイラーが分担する給湯熱負荷(MJ/h)  
$L''_{ba2,d,t}$：日付$d$の時刻$t$における1時間当たりの浴槽追焚時の太陽熱補正給湯熱負荷(MJ/h)  

In [ ]:
# 1時間当たりの浴槽追焚時におけるバックアップボイラーが分担する給湯熱負荷 (MJ/h) (28)
def get_L_BB_DHW_ba2_d_t(L_dashdash_ba2_d_t):
    return L_dashdash_ba2_d_t

### 16. その他

$$\large
\displaystyle
L_{DHW,d} = \sum^{23}_{t=0} L_{DHW,d,t}
$$

$L_{DHW,d}$：日付$d$における1日当たりの発電ユニットにおける浴槽追焚を除く給湯熱負荷(MJ/d)  
$L_{DHW,d,t}$：日付$d$の時刻$t$における1 時間当たりの発電ユニットにおける浴槽追焚を除く給湯熱負荷(MJ/h)  

In [ ]:
# 1日当たりの発電ユニットによる浴槽追焚を除く給湯熱負荷 (MJ/d) (29)
def get_L_DHW_d(L_DHW_d_t):
    """
    :param L_DHW_d_t: 1日当たりの発電ユニットにおける浴槽追焚を除く給湯熱負荷 (MJ/d)
    :return: 1日当たりの発電ユニットにおける浴槽追焚を除く給湯熱負荷 (MJ/d)
    """
    L_DHW_d = np.sum(L_DHW_d_t.reshape(365, 24), axis=1)
    return L_DHW_d

$$\large
\displaystyle
L_{DHW,d,t} = L''_{k,d,t} + L''_{w,d,t} + L''_{s,d,t} + L''_{b1,d,t} + L''_{b2,d,t} + L''_{ba1,d,t}
$$

$ L''_{k,d,t}$：日付$d$の時刻$t$における1 時間当たりの台所水栓の太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{w,d,t}$：日付$d$の時刻$t$における1 時間当たりの洗面水栓の太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{s,d,t}$：日付$d$の時刻$t$における1 時間当たりの浴室シャワー水栓の太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b1,d,t}$：日付$d$の時刻$t$における1 時間当たりの浴槽水栓湯はりの太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b2,d,t}$：日付$d$の時刻$t$における1 時間当たりの浴槽自動湯はりの太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba1,d,t}$：日付$d$の時刻$t$における1 時間当たりの浴槽水栓さし湯の太陽熱補正給湯熱負荷(MJ/h)  

In [ ]:
# 1時間当たりの発電ユニットによる浴槽追焚を除く給湯熱負荷 (30)
def get_L_DHW_d_t(L_dashdash_k_d_t, L_dashdash_w_d_t, L_dashdash_s_d_t, L_dashdash_b1_d_t, L_dashdash_b2_d_t,
                  L_dashdash_ba1_d_t):
    """
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりの発電ユニットによる浴槽追焚を除く給湯熱負荷 (MJ/h)
    """
    L_DHW_d_t = L_dashdash_k_d_t + L_dashdash_w_d_t + L_dashdash_s_d_t + L_dashdash_b1_d_t + L_dashdash_b2_d_t + L_dashdash_ba1_d_t
    return L_DHW_d_t

$$\large
\displaystyle
L_{HWH,d} = \sum^{23}_{t=0}L_{HWH,d,t}
$$

$L_{HWH,d,t}$：日付$d$の時刻$t$における1 時間当たりの温水暖房の熱負荷(MJ/h)  

In [ ]:
# 1日当たりの温水暖房の熱負荷 (31)
def get_L_HWH_d(L_HWH_d_t):
    """
    :param L_HWH_d_t: 1時間当たりの温水暖房の熱負荷 (MJ/h)
    :return: 1日当たりの温水暖房の熱負荷 (MJ/d)
    """
    # 8760時間の1次元配列を364x24の2次元配列に変換する
    tmp = L_HWH_d_t.reshape((365, 24))

    # 2次元目を合算して配列を1次元化
    L_HWH_d = np.sum(tmp, axis=1)

    return L_HWH_d

## 付録 A ~ C

省略